In [1]:
# Checking once more
from skimage import io
import cupy as cp
import numpy as np
from skimage.measure import regionprops
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import time
start_time = time.time()

# Charger l'image TIFF
image = io.imread('labeled3D_with_time_image.ome.tif')

# Initialisations
volume_px_dict = {}
surface_px_dict = {}
surface_element_dict = {}
ratio_px_dict = {}

array_3d = np.array([
    [[0, 0, 0], [0, 1, 0], [0, 0, 0]],
    [[0, 1, 0], [1, 1, 1], [0, 1, 0]],
    [[0, 0, 0], [0, 1, 0], [0, 0, 0]]
],dtype=np.float64)

m_depth, m_height, m_width = array_3d.shape
print(array_3d.shape)


# ###################
#     for z in range(1, slices_at_t.shape[0] - m_depth + 1):
#         for y in range(1, slices_at_t.shape[1] - m_height + 1):
#             for x in range(1, slices_at_t.shape[2] - m_width + 1):
#                 # Effectuer la multiplication élément par élément
#                 element_wise_product = slices_at_t[z:z+m_depth, y:y+m_height, x:x+m_width] * array_3d
                
#                 #print("checking element wise shape",element_wise_product.shape)
                
#                 # Définir la valeur du pixel central
#                 px = element_wise_product[1, 1, 1]
#                 if px != 0 and cp.count_nonzero(element_wise_product == px) > 1 and cp.count_nonzero(element_wise_product != px) > 20:
#                     #print("I am in doubtful part")
#                     surface_element_dict[int(px)] += 1
# #################

                             
# CuPy kernel for surface element computation
surface_kernel = cp.RawKernel(r'''
extern "C" __global__
void compute_surface_elements(
    double* slices_at_t, double* surface_element_array_gpu, double* array_3d,
    const int m_depth, const int m_height, const int m_width,
    int slices_shape_0, int slices_shape_1, int slices_shape_2) 
{
    int z = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    int x = blockIdx.z * blockDim.z + threadIdx.z;
    
    int small_cube_z, small_cube_y, small_cube_x;

    // As per code, start is from one.
    int start_z = 1, start_y = 1, start_x = 1; 

    int end_z, end_y, end_x;
    int linear_id_big_cube;
    int linear_id_small_cube;
    int array_3d_linear_id;
    
    double px;
    
    int equal_px_count = 0;
    int unequal_px_count = 0;

    double element_wise_product[3][3][3];
    //printf("%d, %d, %d", m_depth, m_height, m_width);
    
    //float *element_wise_product = new float[m_depth * m_height * m_width];

    end_z = slices_shape_0 - m_depth + 1;
    end_y = slices_shape_1 - m_height + 1;
    end_x = slices_shape_2 - m_width + 1;

    //printf("going to compute \n");
    
    //printf("small cube z = %f, sum = %f \n", z);
    
    if (z >= start_z && z < end_z && y >= start_y && y < end_y && x >= start_x && x < end_x) 
    {
        for (int dz = 0; dz < m_depth; dz++) 
        {   
            small_cube_z = z + dz;
            
            for (int dy = 0; dy < m_height; dy++) 
            {   
                small_cube_y = y + dy;
                for (int dx = 0; dx < m_width; dx++) 
                {   
                    
                    small_cube_x = x + dx;
                    //printf("dx =  %d \n", dx);
                    
                    array_3d_linear_id = dz * m_height * m_width + dy * m_width + dx;
                        
                    linear_id_big_cube = small_cube_z * slices_shape_1 * slices_shape_2 + small_cube_y * slices_shape_2 + small_cube_x;
                    
                    //printf("array 3d linear id = %d, val = %f \n", array_3d_linear_id, array_3d[array_3d_linear_id]);
                    //printf("z, y, x = %d %d %d \n", small_cube_z, small_cube_y, small_cube_x);
                     
                    element_wise_product[dz][dy][dx] = slices_at_t[linear_id_big_cube] * array_3d[array_3d_linear_id];
                    
                    //printf("linear id, value  %f  %f \n", array_3d_linear_id, element_wise_product[array_3d_linear_id]);
                }
            }
        }
        
        //printf("complete comp loop \n");
        
        //printf("problematic element %f \n", element_wise_product[13]);
        px = element_wise_product[1][1][1];
        
        for (int dz = 0; dz < m_depth; dz++) 
        {   
            for (int dy = 0; dy < m_height; dy++) 
            {   
                for (int dx = 0; dx < m_width; dx++) 
                {   
                    //printf("px = %f, element =   %f \n", px,element_wise_product[dz][dy][dx]);
                    if(element_wise_product[dz][dy][dx] == px)
                    {
                        equal_px_count += 1;
                        //printf("True ");
                    }
                    else
                    {
                        unequal_px_count += 1;
                        //printf("Unequal True ");
                    }
                }
            }
        }
        
        
         //printf("starting binning \n");
        
        if(px!=0 && equal_px_count > 1 && unequal_px_count > 20)
        {
            atomicAdd(&surface_element_array_gpu[int(px)], 1);
            //printf(" This is multiple conditonal and px = %f \n", px);
        }

    }
}
''', 'compute_surface_elements')


# Supposons que 'label_image' est votre image étiquetée 3D
# for t in range(image.shape[1]): 
for t in range(3): 
    # Accéder aux données pour la valeur actuelle de t
    slices_at_t = image[:, t, :, :]
    #print(slices_at_t)
    slices_at_t = cp.asarray(slices_at_t)  # Convertir en tableau CuPy'
    
    print(type(slices_at_t));
    print(slices_at_t.device);
    print("shape of slice array is = ", slices_at_t.shape)
    print("Slices_at_t dtype = ", slices_at_t.dtype)
    print("Slices_at_t bytes = ", slices_at_t.nbytes)


    
    array_3d_gpu = cp.array(array_3d)
    #array_3d_gpu = array_3d_gpu.astype(cp.float64)
    print(type(array_3d_gpu))
    print(array_3d_gpu.device)
    print("Array 3d dtype = ",array_3d_gpu.dtype)
    print("Array 3d bytes = ",array_3d_gpu.nbytes)
    print("Array 3d gpu is = ", array_3d_gpu)
    
    # Pour le volume
    volume_px_dict[f'vol_pix_at_t={t}'] = []
    properties = regionprops(cp.asnumpy(slices_at_t))
    for prop in properties:
        volume_px_dict[f'vol_pix_at_t={t}'].append((prop.label, prop.area))
    
    # Pour l'aire de surface
    surface_px_dict[f'surf_pix_at_t={t}'] = []
    unique_values = cp.unique(slices_at_t)
    unique_values = unique_values[unique_values != 0]
    for value in unique_values:
        surface_element_dict[int(value)] = 0

    # we will use this to create gpu array to update atomically.    
    # This is binning without maps.
    max_value = np.max(unique_values)
    
    
    #print("number of unique values ", unique_values, unique_values.shape)
    # create array on gpu
    # This will work only if keys arae in the range unique_values.
    surface_element_array_gpu = cp.zeros(int(max_value)+1,dtype=cp.float64)
    
    # Pour le ratio SA:V
    ratio_px_dict[f'ratio_at_t={t}'] = []
    
    print("z = ", slices_at_t.shape[0] - m_depth + 1)
    print("y = ", slices_at_t.shape[1] - m_height + 1)
    print("x = ", slices_at_t.shape[2] - m_width + 1)

    slices_at_t = slices_at_t.astype(cp.float64)
    
    # Launch the kernel
    threads_per_block = (8, 8, 8)
    blocks_per_grid = (
        (slices_at_t.shape[0]) // threads_per_block[0] + 1,
        (slices_at_t.shape[1]) // threads_per_block[1] + 1,
        (slices_at_t.shape[2]) // threads_per_block[2] + 1
    )
    #blocks_per_grid = (1,1,1)
    surface_kernel(
        blocks_per_grid, threads_per_block,
        (slices_at_t, surface_element_array_gpu, array_3d_gpu,
         m_depth, m_height, m_width,
         slices_at_t.shape[0], slices_at_t.shape[1], slices_at_t.shape[2])
    )
    
    # Ensure synchronization and check for errors
    cp.cuda.runtime.deviceSynchronize()   
    
    # Get array back (.get function having issues)
    surface_element_array_cpu = cp.asnumpy(surface_element_array_gpu)
    # print(type(surface_element_array_cpu))
    print("surface elements array cpu = ",surface_element_array_cpu.shape)
    #print(surface_element_array_cpu)
    
    # Copy into dictionary array
    for value in unique_values:
         surface_element_dict[int(value)] = surface_element_array_cpu[int(value)]
                    
    for value in unique_values:
        surface_px_dict[f'surf_pix_at_t={t}'].append((int(value), surface_element_dict[int(value)]))

    ratio_px_dict[f'ratio_at_t={t}'] = [(SA[0], SA[1] / V[1]) for SA, V in zip(surface_px_dict[f'surf_pix_at_t={t}'], volume_px_dict[f'vol_pix_at_t={t}'])]

print("longueur :", len(surface_px_dict[f'surf_pix_at_t={2}']))
print(volume_px_dict[f'vol_pix_at_t={2}'])
print(surface_px_dict[f'surf_pix_at_t={2}'])
print(ratio_px_dict[f'ratio_at_t={2}'])


end_time = time.time()

execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

(3, 3, 3)
<class 'cupy.ndarray'>
<CUDA Device 0>
shape of slice array is =  (201, 320, 300)
Slices_at_t dtype =  uint16
Slices_at_t bytes =  38592000
<class 'cupy.ndarray'>
<CUDA Device 0>
Array 3d dtype =  float64
Array 3d bytes =  216
Array 3d gpu is =  [[[0. 0. 0.]
  [0. 1. 0.]
  [0. 0. 0.]]

 [[0. 1. 0.]
  [1. 1. 1.]
  [0. 1. 0.]]

 [[0. 0. 0.]
  [0. 1. 0.]
  [0. 0. 0.]]]
z =  199
y =  318
x =  298
surface elements array cpu =  (287,)
<class 'cupy.ndarray'>
<CUDA Device 0>
shape of slice array is =  (201, 320, 300)
Slices_at_t dtype =  uint16
Slices_at_t bytes =  38592000
<class 'cupy.ndarray'>
<CUDA Device 0>
Array 3d dtype =  float64
Array 3d bytes =  216
Array 3d gpu is =  [[[0. 0. 0.]
  [0. 1. 0.]
  [0. 0. 0.]]

 [[0. 1. 0.]
  [1. 1. 1.]
  [0. 1. 0.]]

 [[0. 0. 0.]
  [0. 1. 0.]
  [0. 0. 0.]]]
z =  199
y =  318
x =  298
surface elements array cpu =  (287,)
<class 'cupy.ndarray'>
<CUDA Device 0>
shape of slice array is =  (201, 320, 300)
Slices_at_t dtype =  uint16
Slices_at_t by

In [5]:
z =  199
y =  318
x =  298
print(x*y*z)


18858036


In [2]:
speedup = 106/1.4
print(speedup)

75.71428571428572
